# Import packages

In [3]:
import numpy as np

# Chart modules
import matplotlib.pyplot as plt
from bokeh.plotting import figure
from bokeh.io import push_notebook,show, output_notebook
from bokeh.models import HoverTool
output_notebook()

from modules.Utils.utils import loadFromDB, strategyTester, fourrierExtrapolation
import pandas as pd

Loading BokehJS ...

In [31]:
SYMBOL= 'BTC'
index = 35

df = loadFromDB(f'./Data/15m/{SYMBOL}-USDT.csv').iloc[-index*100:-(index-2)*100]

In [32]:

pred_count = 20
train_data = df.Close[:-pred_count]
test_data = df.Close[-pred_count:]

x = np.array(train_data)

n_predict = pred_count
extrapolation = fourrierExtrapolation(x, n_predict)

p = figure(title=f"Prediction on {SYMBOL} from BTC", x_axis_label='Time', y_axis_label='value ($)',width=1500, height=600,)
# Add the HoverTool to the figure
p.add_tools(HoverTool(
    tooltips=[
    ("Price", "@y{0.00} $"),
    #("Date", "$x{%F}"),
], formatters={
        '$x': 'printf',
        '$y' : 'printf',
    },
    mode='vline'
))
p.line(np.arange(0, x.size), x, legend_label="Train data", line_width=2,)
p.line(np.arange(0, extrapolation.size), extrapolation,legend_label="Extrapolation", line_width=2, color='orange')
p.line(np.arange(x.size, extrapolation.size), test_data,legend_label="Test data", line_width=2, color='black')
p.legend.location = "top_left"

show(p)